In [1]:
%load_ext watermark
%watermark -a 'Scott Ming' -v -m -d -p numpy

Scott Ming 2017-05-30 

CPython 3.6.0
IPython 6.0.0

numpy 1.12.1

compiler   : GCC 4.9.2
system     : Linux
release    : 3.16.0-4-amd64
machine    : x86_64
processor  : 
CPU cores  : 8
interpreter: 64bit


## 测试题 1

在多元线性回归问题中，假设在各个x和y之间存在如下的线性关系：
y = w0 + w1 * x1 + w2 * x2 + w3 * x3
需要去估计各自变量的权重系数w。请用 Python 写出一段代码，来计算出权重。
要求：
- 数据采用附件中的data.csv
- 工具使用 Python 以及 NumPy 库，不能使用其它扩展库
- 写出三种不同的算法
    + 一种是使用有显式解析解的矩阵方法
    + 一种是使用牛顿法
    + 一种是使用随机梯度下降算法
- 分别检验不同算法的模型参数在显著水平为 0.05 时的效果
- 需要有简单的说明文档和代码注释

### 1.0 读取数据

In [2]:
import numpy as np
import scipy.stats as st  # 仅用来计算 p- 值

In [3]:
data = np.genfromtxt('data.csv', delimiter=',', skip_header=1)
data.shape

(1000, 4)

### 1.1 最小二乘法

$$\beta = (X^TX)^{-1}X^Ty$$

$$SSE = \sum_{i=1}^N(y_i - \hat{y})^2$$

$$\sigma^2 = MSE = \frac{SSE}{n - p - 1}$$

$$C = cov(\beta) = \sigma^2(XX^T)^{-1}$$

$$SE = \sqrt{C}$$

$$t_i = \frac{\beta_i}{SE_{i,i}}$$

In [4]:
def compute_p(X, y, w_, predict):
    dof = X.shape[0] - X.shape[1]  # Degree of freedom
    sse = np.sum((predict - y) ** 2, axis=0)
    mse = sse / dof
    c = mse * np.linalg.inv(X.T.dot(X))
    se = np.sqrt(np.diag(c))  # 取对角线的元素
    t = w_ / se
    p_ = 2 * (1 - st.t.cdf(np.abs(t), dof))
    return t, p_


def print_p_values(p_):
    x = 'b0 b1 b2 b3'.split()
    for x, p in zip(x, p_):
        print(f'{x}: {p:.20f}')

In [5]:
class LinearRegressionLS(object):
    def fit(self, X, y):
        self.w_ = np.linalg.lstsq(X, y)[0]
        self.t_, self.p_ = compute_p(X, y, self.w_, self.predict(X))
        return self
    
    def net_input(self, X):
        return np.dot(X[:, 1:], self.w_[1:]) + self.w_[0]
    
    def predict(self, X):
        return self.net_input(X)  

In [6]:
X = data[:, :3]
X = np.c_[np.ones(X.shape[0]).T, X]  # 添加一个截距项在左边
y = data[:, 3]

In [7]:
ls = LinearRegressionLS()
ls.fit(X, y)
ls.w_

array([ 2.03076198,  2.97396653, -0.54139002,  0.97132913])

**显著性检验：**

$$H_0 :\beta_i = 0$$
$$H_a :\beta_i \neq 0$$

In [8]:
print_p_values(ls.p_)

b0: 0.00000000000000000000
b1: 0.00000000000000000000
b2: 0.00000001179840070087
b3: 0.00000000000000000000


从上面的打印结果，可以看出，各 $\beta$ 的 $p-$ 值远小于 0.05，于是拒绝原假设，$x_i$ 与 $y$ 有显著关系

### 1.2 牛顿法

参考 PRML 4.3.3 节


$w^{(new)} = w^{(old)} - H^{-1}\bigtriangledown E(w) \tag{1.2.1}$

$\bigtriangledown E(w) = \sum_{n=1}^{N}(w^T\phi_n - t_n)\phi_n = \Phi^T\Phi w - \Phi^Tt \tag{1.2.2}$

$H = \bigtriangledown \bigtriangledown E(w) = \sum_{n=1}^{N}\phi_n\phi_n^T = \Phi^T\Phi \tag{1.2.3}$

\begin{align}
w^{(new)} &= w^{(old)} - (\Phi^T\Phi)^{-1}\{\Phi^T\Phi w^{(old)} -\Phi^Tt \} \\
&= (\Phi^T\Phi)^{-1}\Phi^Tt \tag{1.2.4}
\end{align}

从上面公式可以看出，w 迭代一次就会变成最小二乘解

In [9]:
class LinearRegressionNR(object):
    """Newton-Raphoson Method."""
    def __init__(self, w=[0, 0, 0, 0]):
        self.w = w
        
    def fit(self, X, y):
        # Iterate one step 
        self.w = self.w - np.linalg.inv(X.T.dot(X)).dot(
            X.T.dot(X).dot(self.w) - X.T.dot(y))
        self.t_, self.p_ = compute_p(X, y, self.w, self.predict(X))
        return self
    
    def net_input(self, X):
        return np.dot(X[:, 1:], self.w[1:]) + self.w[0]
    
    def predict(self, X):
        return self.net_input(X) 

In [10]:
nr = LinearRegressionNR()

这里 `w` 因为被 `__init__` 初始化，所以后面不加 `_` 线

In [11]:
# 初始化的 w
nr.w

[0, 0, 0, 0]

迭代一次，拟合之后

In [12]:
nr.fit(X, y)
nr.w

array([ 2.03076198,  2.97396653, -0.54139002,  0.97132913])

In [13]:
print_p_values(nr.p_)

b0: 0.00000000000000000000
b1: 0.00000000000000000000
b2: 0.00000001179840070087
b3: 0.00000000000000000000


**显著性检验:** 因为结果与最小二乘法是完全一样的，所以检验结果也一样，不再重复

### 1.3 随机梯度下降法

随机梯度下降，我在 DL101 课程上有多次实作过，用 Numpy 实作其实也很简单，理解最小化成本函数和更新规则即可，最近刚好在读 Python Machine Learning 这本书，ch2 的关于 **AdLine 分类模型**的代码写的非常漂亮，而线性回归和 AdLine 两者的成本函数其实是完全一样的，两者都是用误差平放和(SSE) 做优化，所以我直接改了这个类。

$$J(w) = \frac{1}{2}\sum_i(y^{(i)} - \phi{(z^{(i)})})^2$$

$$\frac{\delta J}{\delta w_j} = \frac{\delta}{\delta w_j}\frac{1}{2}(y^{(i)} - \phi{(z^{(i)})})^2 = -\sum_i(y^{(i)} - \phi(z^{i}))x_j^{(i)}$$

$$\Delta w = -\eta\frac{\delta J}{\delta w_j}  = \eta\sum_i(y^{(i)} - \phi{(z^{(i)})})x_j^{(i)}$$

$$ w := w + \Delta w$$

In [14]:
class LinearRegressionSGD(object):
    """Linear Regression.

    Parameters
    ------------
    eta : float
        Learning rate (between 0.0 and 1.0)
    n_iter : int
        Passes over the training dataset.

    Attributes
    -----------
    w_ : 1d-array
        Weights after fitting.
    cost_ : list
        Sum-of-squares cost function value averaged over all
        training samples in each epoch.
    shuffle : bool (default: True)
        Shuffles training data every epoch if True to prevent cycles.
    random_state : int (default: None)
        Set random state for shuffling and initializing the weights.
        
    """
    def __init__(self, eta=0.01, n_iter=20, shuffle=True, random_state=None):
        self.eta = eta
        self.n_iter = n_iter
        self.w_initialized = False
        self.shuffle = shuffle
        if random_state:
            np.random.seed(random_state)
        
    def fit(self, X, y):
        """ Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like, shape = [n_samples]
            Target values.

        Returns
        -------
        self : object

        """
        self._initialize_weights(X.shape[1])
        self.cost_ = []
        for i in range(self.n_iter):
            if self.shuffle:
                X, y = self._shuffle(X, y)
            cost = []
            # 每个迭代下，打乱数据，然后把每个 xi, target 依次迭代一遍
            for xi, target in zip(X, y):
                cost.append(self._update_weights(xi, target))
            avg_cost = sum(cost) / len(y)
            self.cost_.append(avg_cost)
        self.t_, self.p_ = compute_p(X, y, self.w_, self.predict(X))
        return self

    def _shuffle(self, X, y):
        """Shuffle training data"""
        r = np.random.permutation(len(y))
        return X[r], y[r]
    
    def _initialize_weights(self, m):
        """Initialize weights to zeros"""
        self.w_ = np.zeros(m)
        self.w_initialized = True
        
    def _update_weights(self, xi, target):
        """Apply Adaline learning rule to update the weights"""
        output = self.net_input(xi)
        error = (target - output)
        self.w_ += self.eta * xi.dot(error)
        # 已经添加了截距项，所以不需要另外求 w_[0]
#         self.w_[0] += self.eta * error
        cost = 0.5 * error**2
        return cost
    
    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_)
    
    def predict(self, X):
        return self.net_input(X)

In [15]:
sgd = LinearRegressionSGD(n_iter=50, eta=0.01, random_state=42)

In [16]:
sgd.fit(X, y)
sgd.w_

array([ 1.95317769,  2.90773368, -0.52167386,  0.90497183])

In [17]:
sgd.cost_

[2.2948359275842689,
 1.9363849020929147,
 1.9266239612428337,
 1.9274290834534784,
 1.9272601525159454,
 1.9354518909254301,
 1.9369892657599501,
 1.9374456223217564,
 1.9343635788284617,
 1.9212550680188527,
 1.9362110158731707,
 1.9398331678158576,
 1.9345545865661109,
 1.9341039214781384,
 1.9373954829761602,
 1.920753117075195,
 1.9267799280903986,
 1.9302494972024082,
 1.9274435556714449,
 1.9271440823503754,
 1.9338529693466362,
 1.9309339369588017,
 1.9351922967151389,
 1.9288950460647132,
 1.9369090373622002,
 1.9354932530540114,
 1.9325468439884077,
 1.9368158325949341,
 1.9401035187864792,
 1.9230762752870858,
 1.9307334622416124,
 1.9334926272473543,
 1.9345203043383112,
 1.9304455512762488,
 1.9324758880899173,
 1.9360113462019093,
 1.9345500484691025,
 1.9279644345200377,
 1.9303477731425058,
 1.9289942847341102,
 1.9232761609143567,
 1.9338434365116322,
 1.9298349161646449,
 1.9299474378528692,
 1.9143257490960159,
 1.9345587169091312,
 1.9193866517343212,
 1.93899622875

SGD 算法，模型在第二步就开始收敛了

**显著性检验：**

$$H_0 :\beta_i = 0$$
$$H_a :\beta_i \neq 0$$

In [18]:
print_p_values(sgd.p_)

b0: 0.00000000000000000000
b1: 0.00000000000000000000
b2: 0.00000004052505619967
b3: 0.00000000000000000000


从上面的打印结果，可以看出，各 $\beta$ 的 $p-$ 值远小于 0.05，于是拒绝原假设，$x_i$ 都与 $y$ 有显著关系